## **Amazon SageMaker Model Monitor**
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata
* Analyze a training dataset to generate baseline constraints
* Monitor a live endpoint for violations against constraints

## **Background**

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to analyze the data and monitor its quality. 

## **Setup**

* Specify an AWS Region to host your model.
* An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3). 
* Create an S3 bucket used to store the data used to train your model, any additional model data, and the data captured from model invocations. 

In [1]:
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  'ta-sagemaker-experiments'
#print("Bucket: {}".format(bucket))
prefix = 'ModelMonitor'

data_capture_prefix = '{}/datacapture'.format(prefix)
#s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

code_prefix = '{}/code'.format(prefix)

s3_code_preprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'preprocessor.py')
s3_code_postprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'postprocessor.py')

#print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

RoleArn: arn:aws:iam::171774164293:role/service-role/AmazonSageMaker-ExecutionRole-20200608T073821
Report path: s3://ta-sagemaker-experiments/ModelMonitor/reports
Preproc Code path: s3://ta-sagemaker-experiments/ModelMonitor/code/preprocessor.py
Postproc Code path: s3://ta-sagemaker-experiments/ModelMonitor/code/postprocessor.py


## **PART A. Capturing real-time inference data from Amazon SageMaker endpoints**

### **Invoke the deployed model**

You can now send data to this endpoint to get inferences in real time. Because we already enabled the data capture while deploying endpoint.
The request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location we have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 2 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [2]:
from sagemaker.predictor import RealTimePredictor
import time

endpoint_name='Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint'
predictor = RealTimePredictor(endpoint=endpoint_name,content_type='text/csv')

# get a subset of test data for a quick test

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))


test_samples=[line.rstrip('\n') for line in open('test_sample_X_without_header.csv')]
for sample in test_samples:
    sample=bytes(sample,'utf-8')
    print(predictor.predict(sample))
print("Done!")


Sending test traffic to the endpoint Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint. 
Please wait...
b'350954.28125'
b'131258.703125'
b'175565.171875'
b'128477.21875'
b'136587.65625'
b'145165.78125'
b'129439.3984375'
b'129255.1015625'
b'150203.953125'
b'185763.78125'
b'142064.859375'
b'108565.0546875'
b'104505.1796875'
b'116970.921875'
b'337586.90625'
b'255605.046875'
b'250518.5625'
b'440427.65625'
b'226071.609375'
b'229986.5'
b'209143.640625'
b'223948.671875'
b'284860.5625'
b'333308.5625'
b'308612.1875'
b'294721.03125'
b'314766.9375'
b'211081.375'
b'243620.484375'
b'234679.90625'
b'271492.40625'
b'100796.1796875'
b'81172.5390625'
b'121743.3046875'
b'92102.3671875'
b'74269.75'
b'101910.7421875'
b'86062.953125'
b'73572.0859375'
b'73572.0859375'
b'77531.078125'
b'93484.53125'
Done!


### **View Captured Data**

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl

In [4]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
ModelMonitor/datacapture/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/AllTraffic/2020/08/18/15/56-42-893-249a48dc-b410-4028-af06-c3b1587f9d84.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [5]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-122.23,37.88,41,880,129,322,126,8.3252,NEAR BAY","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"350954.28125","encoding":"CSV"}},"eventMetadata":{"eventId":"b3c720cc-2329-4e3e-8429-2b2791577a06","inferenceTime":"2020-08-18T15:56:42Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-122.26,37.85,50,1120,283,697,264,2.125,NEAR BAY","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"131258.703125","encoding":"CSV"}},"eventMetadata":{"eventId":"f124bd6b-40c4-4567-b741-d7ca2015f3bb","inferenceTime":"2020-08-18T15:56:42Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"-122.27,37.85,52,1228,293,648,303,2.1202,NEAR BAY","encoding":"CSV"},"endpointOutput":{"observedCo

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [6]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "-122.23,37.88,41,880,129,322,126,8.3252,NEAR BAY",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "350954.28125",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "b3c720cc-2329-4e3e-8429-2b2791577a06",
    "inferenceTime": "2020-08-18T15:56:42Z"
  },
  "eventVersion": "0"
}


As you can see, each inference request is captured in one line in the jsonl file.The line contains both the input and output merged together.

## **PART.B. Model Monitor - Baseling and continuous monitoring**

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:

* Create a baseline with which you compare the realtime traffic.
* Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

## **1.Constraint suggestion with baseline/training dataset**

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline constraints and generate descriptive statistics to explore the data. 

upload the training dataset that was used to train the pre-trained model.
If you already have it in Amazon S3, you can directly point to it.

In [7]:
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://ta-sagemaker-experiments/ModelMonitor/baselining/data
Baseline results uri: s3://ta-sagemaker-experiments/ModelMonitor/baselining/results


In [8]:
training_data_file = open("train_data_X_Y_with_header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'train_data_X_Y_with_header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### **Create a baselining job with training dataset**

Now that you have the training data ready in Amazon S3, start a job to suggest constraints. 
DefaultModelMonitor.suggest_baseline(..) starts a ProcessingJob using an Amazon SageMaker provided Model Monitor container to generate the constraints.

In [9]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1200,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/train_data_X_Y_with_header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2020-08-18-15-58-54-646
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://ta-sagemaker-experiments/ModelMonitor/baselining/data/train_data_X_Y_with_header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://ta-sagemaker-experiments/ModelMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................2020-08-18 16:02:49,888 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:171774164293:processing-job/baseline-suggestion-job-2020-08-18-15-58-54-646', 'ProcessingJobName': 'baseline-suggestion-job-2020-08-18-15-58-54-646', 'Environment': {'dataset_format': '{"csv": {"header": true, "output_columns_position": "START"}}', 'd

### **Explore the generated constraints and statistics**

In [10]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)

In [11]:
result

{'ResponseMetadata': {'RequestId': 'BB9D31EFE9F57BE1',
  'HostId': 'H1Vf1kWfraDmubCD8KHeZE8uWx+vGVpv6kI+UwRM3GC0E9yEkAKj9KJ9oD/y+K6t1BKW5IyhgrI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'H1Vf1kWfraDmubCD8KHeZE8uWx+vGVpv6kI+UwRM3GC0E9yEkAKj9KJ9oD/y+K6t1BKW5IyhgrI=',
   'x-amz-request-id': 'BB9D31EFE9F57BE1',
   'date': 'Tue, 18 Aug 2020 16:05:53 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'ModelMonitor/baselining/results/constraints.json',
   'LastModified': datetime.datetime(2020, 8, 18, 16, 4, 1, tzinfo=tzlocal()),
   'ETag': '"2e206b63c3d119a4a1519752ef65dd54"',
   'Size': 2041,
   'StorageClass': 'STANDARD'},
  {'Key': 'ModelMonitor/baselining/results/statistics.json',
   'LastModified': datetime.datetime(2020, 8, 18, 16, 4, 1, tzinfo=tzlocal()),
   'ETag': '"49e133d67ed54a8ca919e18724

In [12]:
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
ModelMonitor/baselining/results/constraints.json
 ModelMonitor/baselining/results/statistics.json


In [13]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data,string_statistics.common.num_present,string_statistics.common.num_missing,string_statistics.distinct_count,string_statistics.distribution.categorical.buckets
0,median_house_value,Integral,16555.0,0.0,207069.964784,3.428043e+09,115418.860329,14999.0000,500001.0000,"[{'lower_bound': 14999.0, 'upper_bound': 63499...",0.64,2048.0,"[[136700.0, 113300.0, 86500.0, 72000.0, 80800....",NaN,NaN,NaN,NaN
1,longitude,Fractional,16555.0,0.0,-119.562481,-1.979357e+06,2.004458,-124.3500,-114.3100,"[{'lower_bound': -124.35, 'upper_bound': -123....",0.64,2048.0,"[[-122.37, -122.36, -122.41, -122.41, -122.39,...",NaN,NaN,NaN,NaN
2,latitude,Fractional,16555.0,0.0,35.628673,5.898327e+05,2.137281,32.5400,41.9500,"[{'lower_bound': 32.54, 'upper_bound': 33.481,...",0.64,2048.0,"[[40.55, 40.55, 40.55, 40.53, 40.53, 40.52, 40...",NaN,NaN,NaN,NaN
3,housing_median_age,Integral,16555.0,0.0,28.563153,4.728630e+05,12.574684,1.0000,52.0000,"[{'lower_bound': 1.0, 'upper_bound': 6.1, 'cou...",0.64,2048.0,"[[26.0, 21.0, 19.0, 28.0, 28.0, 24.0, 20.0, 18...",NaN,NaN,NaN,NaN
4,total_rooms,Integral,16555.0,0.0,2650.804893,4.388408e+07,2205.733496,2.0000,39320.0000,"[{'lower_bound': 2.0, 'upper_bound': 3933.8, '...",0.64,2048.0,"[[1435.0, 2500.0, 3753.0, 1127.0, 1427.0, 2068...",NaN,NaN,NaN,NaN
5,total_bedrooms,Integral,16393.0,162.0,540.488562,8.860229e+06,425.520755,1.0000,6445.0000,"[{'lower_bound': 1.0, 'upper_bound': 645.4, 'c...",0.64,2048.0,"[[1001.0, 159.0, 727.0, 790.0, 1759.0, 1188.0,...",NaN,NaN,NaN,NaN
6,population,Integral,16555.0,0.0,1431.906735,2.370522e+07,1151.505004,3.0000,35682.0000,"[{'lower_bound': 3.0, 'upper_bound': 3570.9, '...",0.64,2048.0,"[[544.0, 1428.0, 1952.0, 538.0, 692.0, 951.0, ...",NaN,NaN,NaN,NaN
7,households,Integral,16555.0,0.0,502.168529,8.313400e+06,387.277490,1.0000,6082.0000,"[{'lower_bound': 1.0, 'upper_bound': 609.1, 'c...",0.64,2048.0,"[[232.0, 502.0, 738.0, 208.0, 285.0, 332.0, 34...",NaN,NaN,NaN,NaN
8,median_income,Fractional,16555.0,0.0,3.884336,6.430519e+04,1.913171,0.4999,15.0001,"[{'lower_bound': 0.4999, 'upper_bound': 1.9499...",0.64,2048.0,"[[2.6705, 2.6513, 3.0954, 2.037, 2.125, 3.9306...",NaN,NaN,NaN,NaN
9,ocean_proximity,String,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16555.0,0.0,5.0,"[{'value': '<1H OCEAN', 'count': 7331}, {'valu..."


In [14]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative,string_constraints.domains
0,median_house_value,Integral,1.000000,True,NaN
1,longitude,Fractional,1.000000,False,NaN
2,latitude,Fractional,1.000000,True,NaN
3,housing_median_age,Integral,1.000000,True,NaN
4,total_rooms,Integral,1.000000,True,NaN
5,total_bedrooms,Integral,0.990214,True,NaN
6,population,Integral,1.000000,True,NaN
7,households,Integral,1.000000,True,NaN
8,median_income,Fractional,1.000000,True,NaN
9,ocean_proximity,String,1.000000,NaN,"[<1H OCEAN, ISLAND, INLAND, NEAR BAY, NEAR OCEAN]"


## **2. Analyzing collected data for data quality issues**

When you have collected the data above, analyze and monitor the data with Monitoring Schedules

### **Create a schedule**

In [15]:
# First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/preprocessor.py").upload_file('preprocessor.py')
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/postprocessor.py").upload_file('postprocessor.py')

In [16]:
from datetime import date,datetime
today = datetime.today()
print("Today's date:", today)

Today's date: 2020-08-18 16:06:13.036416


You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.

In [17]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'Builtin-XGB-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    #record_preprocessor_script='preprocessor.py',
    post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    #schedule_cron_expression="cron(0 19/1 ? * * *)",
    enable_cloudwatch_metrics=True,

)

#Today's date: 2020-08-17 03:44:24.395104


Creating Monitoring Schedule with name: Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38


In [ ]:
#my_default_monitor.delete_monitoring_schedule()

### **Start generating some artificial traffic**

The cell below starts a thread to send some traffic to the endpoint.

Note that you need to stop the kernel to terminate this thread(if while=True). If there is no traffic, the monitoring jobs are marked as Failed since there is no data to process.

As of now we have set some limited iterations

In [18]:
from threading import Thread
from time import sleep
import time

endpoint_name=predictor.endpoint
runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
        test_samples=[line.rstrip('\n') for line in open(file_name)]
        for sample in test_samples:
            sample=bytes(sample,'utf-8')
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                      ContentType='text/csv', 
                                      Body=sample)
            time.sleep(1)
            
def invoke_endpoint_forever():
    #while True:
    for i in range(60):
        invoke_endpoint(endpoint_name, 'test_sample_X_without_header.csv', runtime_client)
        time.sleep(60)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

### **Describe and inspect the schedule**

Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [19]:
desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))


Schedule status: Scheduled


In [20]:
desc_schedule_result = my_default_monitor.describe_schedule()
print(desc_schedule_result)

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:171774164293:monitoring-schedule/builtin-xgb-model-monitor-schedule-2020-08-18-16-14-38', 'MonitoringScheduleName': 'Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38', 'MonitoringScheduleStatus': 'Scheduled', 'CreationTime': datetime.datetime(2020, 8, 18, 16, 14, 38, 782000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 8, 18, 16, 14, 46, 326000, tzinfo=tzlocal()), 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'}, 'MonitoringJobDefinition': {'BaselineConfig': {'ConstraintsResource': {'S3Uri': 's3://ta-sagemaker-experiments/ModelMonitor/baselining/results/constraints.json'}, 'StatisticsResource': {'S3Uri': 's3://ta-sagemaker-experiments/ModelMonitor/baselining/results/statistics.json'}}, 'MonitoringInputs': [{'EndpointInput': {'EndpointName': 'Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint', 'LocalPath': '/opt/ml/processing/input/endpoint', 'S3InputMode': 'File', 'S3Da

### **List executions**

The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [32]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()

We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...


In [ ]:
mon_executions

In [33]:
my_default_monitor.list_executions()

### **Inspect a specific execution (latest execution)**


Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean:

* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
*Stopped - job exceeded max runtime or was manually stopped.

In [34]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 5 violations.


In [35]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

Report Uri: s3://ta-sagemaker-experiments/ModelMonitor/reports/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38/2020/08/18/17


### **List the generated reports**

In [36]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: ta-sagemaker-experiments
Report key: ModelMonitor/reports/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38/2020/08/18/17
Found Report Files:
ModelMonitor/reports/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38/2020/08/18/17/constraint_violations.json
 ModelMonitor/reports/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38/2020/08/18/17/constraints.json
 ModelMonitor/reports/Builtin-XGB-algo-2020-08-18--11-43-38-Endpoint/Builtin-XGB-model-monitor-schedule-2020-08-18-16-14-38/2020/08/18/17/statistics.json


### **Violations report**

If there are any violations compared to the baseline, they will be listed here.

In [37]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,feature_name,constraint_check_type,description
0,median_income,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 97.61904761904762% of data is Fractional."
1,median_house_value,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,longitude,baseline_drift_check,Baseline drift distance: 0.3986102370000586 exceeds threshold: 0.1
3,latitude,baseline_drift_check,Baseline drift distance: 0.25523470228665285 exceeds threshold: 0.1
4,ocean_proximity,baseline_drift_check,Baseline drift distance: 0.17125749771275328 exceeds threshold: 0.1


### **Other commands**

We can also start and stop the monitoring schedules.

In [ ]:
#my_default_monitor.stop_monitoring_schedule()
#my_default_monitor.start_monitoring_schedule()

## **Delete the resources**

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(60) # actually wait for the deletion

In [ ]:
predictor.delete_endpoint()

In [ ]:
predictor.delete_model()